In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '58 tGravityAcc-energy()-Y',
 '59 tGravityAcc-energy()-Z',
 '128 tBodyGyro-mad()-Y',
 '141 tBodyGyro-iqr()-Y',
 '428 fBodyGyro-std()-Y',
 '434 fBodyGyro-max()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '203 tBodyAccMag-mad()',
 '215 tGravityAccMag-std()',
 '216 tGravityAccMag-mad()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '382 fBodyAccJerk-bandsEnergy()-1,8',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 30),
            classifier_block(30, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            nn.Linear(15, 12)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../../saved_models/{model_name}'))

# Train on Real Test on Real

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_1 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_2 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_3 = GAN_data[(GAN_data['Subject'] == 1) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_4 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_5 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_6 = GAN_data[(GAN_data['Subject'] == 3) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_7 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_8 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_9 = GAN_data[(GAN_data['Subject'] == 5) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    X_10 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 1)].iloc[:,:-2].values
    X_11 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 3)].iloc[:,:-2].values
    X_12 = GAN_data[(GAN_data['Subject'] == 7) & (GAN_data['Activity'] == 4)].iloc[:,:-2].values
    
    X_train = np.concatenate((X_1, X_2, X_3, X_4, X_5, X_6, X_7, X_8, X_9, X_10, X_11, X_12))
    y_train = [0] * len(X_1) + [1] * len(X_2) + [2] * len(X_3) + [3] * len(X_4) + [4] * len(X_5) + [5] * len(X_6) + [6] * len(X_7) + [7] * len(X_8) + [8] * len(X_9) + [9] * len(X_10) + [10] * len(X_11) + [11] * len(X_12)
    
    return X_train, np.asarray(y_train)

In [6]:
activities = [1, 3, 4]
users = [1, 3, 5, 7]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [8]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 7.4565019607543945, Final Batch Loss: 2.4861161708831787
Epoch 2, Loss: 7.389077186584473, Final Batch Loss: 2.4213759899139404
Epoch 3, Loss: 7.438206195831299, Final Batch Loss: 2.476147413253784
Epoch 4, Loss: 7.425340890884399, Final Batch Loss: 2.4706366062164307
Epoch 5, Loss: 7.407440423965454, Final Batch Loss: 2.455002784729004
Epoch 6, Loss: 7.455641031265259, Final Batch Loss: 2.507023572921753
Epoch 7, Loss: 7.404532432556152, Final Batch Loss: 2.455768346786499
Epoch 8, Loss: 7.409439325332642, Final Batch Loss: 2.464600086212158
Epoch 9, Loss: 7.442714214324951, Final Batch Loss: 2.5060012340545654
Epoch 10, Loss: 7.377971410751343, Final Batch Loss: 2.4446983337402344
Epoch 11, Loss: 7.463738679885864, Final Batch Loss: 2.538994312286377
Epoch 12, Loss: 7.381815671920776, Final Batch Loss: 2.4655027389526367
Epoch 13, Loss: 7.37760591506958, Final Batch Loss: 2.46934175491333
Epoch 14, Loss: 7.332941293716431, Final Batch Loss: 2.4351930618286133
Epoch 15,

Epoch 121, Loss: 3.4227999448776245, Final Batch Loss: 1.2566736936569214
Epoch 122, Loss: 3.445485472679138, Final Batch Loss: 1.2329602241516113
Epoch 123, Loss: 3.1802684664726257, Final Batch Loss: 0.9533560872077942
Epoch 124, Loss: 3.2421780824661255, Final Batch Loss: 0.9654021263122559
Epoch 125, Loss: 3.391984462738037, Final Batch Loss: 1.1992532014846802
Epoch 126, Loss: 3.08952397108078, Final Batch Loss: 0.9206394553184509
Epoch 127, Loss: 3.0704381465911865, Final Batch Loss: 0.9501303434371948
Epoch 128, Loss: 3.337642788887024, Final Batch Loss: 1.1260581016540527
Epoch 129, Loss: 3.213936448097229, Final Batch Loss: 1.022255301475525
Epoch 130, Loss: 3.31439745426178, Final Batch Loss: 1.1407952308654785
Epoch 131, Loss: 3.3022472858428955, Final Batch Loss: 1.132822036743164
Epoch 132, Loss: 3.1774948835372925, Final Batch Loss: 0.9923214912414551
Epoch 133, Loss: 3.4500350952148438, Final Batch Loss: 1.2308024168014526
Epoch 134, Loss: 3.1075358390808105, Final Batch

Epoch 232, Loss: 2.794485032558441, Final Batch Loss: 1.0647879838943481
Epoch 233, Loss: 2.5321959853172302, Final Batch Loss: 0.786387026309967
Epoch 234, Loss: 1.9754333198070526, Final Batch Loss: 0.3347020447254181
Epoch 235, Loss: 2.54498028755188, Final Batch Loss: 0.8374310731887817
Epoch 236, Loss: 2.537276029586792, Final Batch Loss: 0.9168505668640137
Epoch 237, Loss: 2.389365315437317, Final Batch Loss: 0.6080032587051392
Epoch 238, Loss: 2.187043458223343, Final Batch Loss: 0.4386216104030609
Epoch 239, Loss: 2.3405430912971497, Final Batch Loss: 0.7358778119087219
Epoch 240, Loss: 2.3713969588279724, Final Batch Loss: 0.7175435423851013
Epoch 241, Loss: 2.6247350573539734, Final Batch Loss: 0.8955686688423157
Epoch 242, Loss: 2.455954134464264, Final Batch Loss: 0.7595963478088379
Epoch 243, Loss: 2.3565760254859924, Final Batch Loss: 0.739057719707489
Epoch 244, Loss: 2.495826303958893, Final Batch Loss: 0.7519140839576721
Epoch 245, Loss: 2.4173234701156616, Final Batch

Epoch 351, Loss: 2.013450562953949, Final Batch Loss: 0.6400045156478882
Epoch 352, Loss: 2.1342252492904663, Final Batch Loss: 0.6904661655426025
Epoch 353, Loss: 2.0439324378967285, Final Batch Loss: 0.5866657495498657
Epoch 354, Loss: 1.9390186667442322, Final Batch Loss: 0.5529820322990417
Epoch 355, Loss: 1.9993765950202942, Final Batch Loss: 0.5824297666549683
Epoch 356, Loss: 1.7692166864871979, Final Batch Loss: 0.2674235999584198
Epoch 357, Loss: 2.4637480974197388, Final Batch Loss: 0.9410055875778198
Epoch 358, Loss: 2.076961040496826, Final Batch Loss: 0.7111289501190186
Epoch 359, Loss: 2.3985010385513306, Final Batch Loss: 0.9990618824958801
Epoch 360, Loss: 1.955083817243576, Final Batch Loss: 0.4821673333644867
Epoch 361, Loss: 1.8687787353992462, Final Batch Loss: 0.4787910282611847
Epoch 362, Loss: 2.514367163181305, Final Batch Loss: 1.039547324180603
Epoch 363, Loss: 1.9633955359458923, Final Batch Loss: 0.6130227446556091
Epoch 364, Loss: 2.2619903683662415, Final 

Epoch 470, Loss: 1.9586249589920044, Final Batch Loss: 0.6220993399620056
Epoch 471, Loss: 1.8814269304275513, Final Batch Loss: 0.5689801573753357
Epoch 472, Loss: 1.8783403038978577, Final Batch Loss: 0.5932577848434448
Epoch 473, Loss: 2.3281983733177185, Final Batch Loss: 1.061290979385376
Epoch 474, Loss: 2.292081296443939, Final Batch Loss: 0.9729374051094055
Epoch 475, Loss: 2.084551751613617, Final Batch Loss: 0.7967334985733032
Epoch 476, Loss: 1.7897415161132812, Final Batch Loss: 0.5070258975028992
Epoch 477, Loss: 1.723101019859314, Final Batch Loss: 0.4327154755592346
Epoch 478, Loss: 1.5935910940170288, Final Batch Loss: 0.3171360492706299
Epoch 479, Loss: 2.260531723499298, Final Batch Loss: 0.991112470626831
Epoch 480, Loss: 1.7151660025119781, Final Batch Loss: 0.4271680414676666
Epoch 481, Loss: 1.959616482257843, Final Batch Loss: 0.6650979518890381
Epoch 482, Loss: 1.8413841128349304, Final Batch Loss: 0.5836924910545349
Epoch 483, Loss: 2.0854429602622986, Final Ba

...

Epoch 4539, Loss: 0.5666025131940842, Final Batch Loss: 0.22004473209381104
Epoch 4540, Loss: 0.5003958716988564, Final Batch Loss: 0.11072828620672226
Epoch 4541, Loss: 0.4873794764280319, Final Batch Loss: 0.12752194702625275
Epoch 4542, Loss: 0.5282069146633148, Final Batch Loss: 0.15377716720104218
Epoch 4543, Loss: 0.4560009315609932, Final Batch Loss: 0.0675114169716835
Epoch 4544, Loss: 0.4628240242600441, Final Batch Loss: 0.0927400216460228
Epoch 4545, Loss: 0.601320207118988, Final Batch Loss: 0.25336387753486633
Epoch 4546, Loss: 0.65164715051651, Final Batch Loss: 0.31512442231178284
Epoch 4547, Loss: 0.45374777913093567, Final Batch Loss: 0.10328014194965363
Epoch 4548, Loss: 0.47137850522994995, Final Batch Loss: 0.12404219806194305
Epoch 4549, Loss: 0.6847534030675888, Final Batch Loss: 0.32900160551071167
Epoch 4550, Loss: 0.627078041434288, Final Batch Loss: 0.25666651129722595
Epoch 4551, Loss: 0.5005881190299988, Final Batch Loss: 0.06389287114143372
Epoch 4552, Loss

Epoch 4649, Loss: 0.6126088798046112, Final Batch Loss: 0.2501172721385956
Epoch 4650, Loss: 0.46036243438720703, Final Batch Loss: 0.08489848673343658
Epoch 4651, Loss: 0.4388889893889427, Final Batch Loss: 0.06672964245080948
Epoch 4652, Loss: 0.5758350044488907, Final Batch Loss: 0.2345191091299057
Epoch 4653, Loss: 0.5879097431898117, Final Batch Loss: 0.24848179519176483
Epoch 4654, Loss: 0.5226708203554153, Final Batch Loss: 0.12284795939922333
Epoch 4655, Loss: 0.6108836829662323, Final Batch Loss: 0.16355466842651367
Epoch 4656, Loss: 0.7377432882785797, Final Batch Loss: 0.36987796425819397
Epoch 4657, Loss: 0.5062555149197578, Final Batch Loss: 0.10905729979276657
Epoch 4658, Loss: 0.5136840343475342, Final Batch Loss: 0.1542578637599945
Epoch 4659, Loss: 0.6591385900974274, Final Batch Loss: 0.30231955647468567
Epoch 4660, Loss: 0.5002055317163467, Final Batch Loss: 0.16202254593372345
Epoch 4661, Loss: 0.4195764120668173, Final Batch Loss: 0.027504777535796165
Epoch 4662, L

Epoch 4759, Loss: 0.7410129308700562, Final Batch Loss: 0.3482312560081482
Epoch 4760, Loss: 0.7254333198070526, Final Batch Loss: 0.3496089577674866
Epoch 4761, Loss: 0.43782223016023636, Final Batch Loss: 0.08625680953264236
Epoch 4762, Loss: 0.5555392354726791, Final Batch Loss: 0.19565466046333313
Epoch 4763, Loss: 0.5398659259080887, Final Batch Loss: 0.14686493575572968
Epoch 4764, Loss: 0.5527344048023224, Final Batch Loss: 0.1711265593767166
Epoch 4765, Loss: 0.46629560366272926, Final Batch Loss: 0.04795316234230995
Epoch 4766, Loss: 0.5304825454950333, Final Batch Loss: 0.18546436727046967
Epoch 4767, Loss: 0.6671982258558273, Final Batch Loss: 0.22338338196277618
Epoch 4768, Loss: 0.7168437242507935, Final Batch Loss: 0.3494672477245331
Epoch 4769, Loss: 0.6112695634365082, Final Batch Loss: 0.22594642639160156
Epoch 4770, Loss: 0.5022335574030876, Final Batch Loss: 0.08229503780603409
Epoch 4771, Loss: 0.5304052531719208, Final Batch Loss: 0.15894407033920288
Epoch 4772, Lo

Epoch 4867, Loss: 0.5132587775588036, Final Batch Loss: 0.10961037129163742
Epoch 4868, Loss: 0.3832574337720871, Final Batch Loss: 0.06140872836112976
Epoch 4869, Loss: 0.48988211154937744, Final Batch Loss: 0.11283731460571289
Epoch 4870, Loss: 0.4494621455669403, Final Batch Loss: 0.06993342936038971
Epoch 4871, Loss: 0.8435440957546234, Final Batch Loss: 0.47234559059143066
Epoch 4872, Loss: 0.6603234112262726, Final Batch Loss: 0.25995898246765137
Epoch 4873, Loss: 0.4013790972530842, Final Batch Loss: 0.04509866610169411
Epoch 4874, Loss: 0.4695611819624901, Final Batch Loss: 0.1002378985285759
Epoch 4875, Loss: 0.557216003537178, Final Batch Loss: 0.18812207877635956
Epoch 4876, Loss: 0.5817863792181015, Final Batch Loss: 0.2120371162891388
Epoch 4877, Loss: 0.6012236475944519, Final Batch Loss: 0.1722569763660431
Epoch 4878, Loss: 0.474753275513649, Final Batch Loss: 0.13140808045864105
Epoch 4879, Loss: 0.482304185628891, Final Batch Loss: 0.13370653986930847
Epoch 4880, Loss:

Epoch 4975, Loss: 0.540269136428833, Final Batch Loss: 0.1484324187040329
Epoch 4976, Loss: 0.48319507390260696, Final Batch Loss: 0.08595595508813858
Epoch 4977, Loss: 0.5825638920068741, Final Batch Loss: 0.16121725738048553
Epoch 4978, Loss: 0.6094497814774513, Final Batch Loss: 0.2919441759586334
Epoch 4979, Loss: 0.610106498003006, Final Batch Loss: 0.21196182072162628
Epoch 4980, Loss: 0.7131725549697876, Final Batch Loss: 0.2602177560329437
Epoch 4981, Loss: 0.5090555250644684, Final Batch Loss: 0.1526234894990921
Epoch 4982, Loss: 0.449469979852438, Final Batch Loss: 0.0513029508292675
Epoch 4983, Loss: 0.43913131207227707, Final Batch Loss: 0.11312951892614365
Epoch 4984, Loss: 0.584770455956459, Final Batch Loss: 0.287514328956604
Epoch 4985, Loss: 0.5496720671653748, Final Batch Loss: 0.1941850632429123
Epoch 4986, Loss: 0.40753748267889023, Final Batch Loss: 0.06299030035734177
Epoch 4987, Loss: 0.7426247596740723, Final Batch Loss: 0.4283389449119568
Epoch 4988, Loss: 0.41

In [66]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 14  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  7  0  0  0  0  0  0  0]
 [ 0  0  4  0  0  2  0  0  2  0  0  1]
 [ 0  0  0  0  1  1 11  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  8  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 15  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  6  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  8]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        17
           1    1.00000   1.00000   1.00000        14
           2    0.50000   1.00000   0.66667         4
           3    1.00000   0.93333   0.96552        15
           4    0.77778   1.00000   0.87500         7
           5    0.20000   0.22222   0.21053         9
           6    1.00000   0.84615   0.91667        13
           7    1.00000   1.00000   1.00000         7
           8    0.80000   0.53333   0.64000 

# Train on Fake Test on Real

In [10]:
# 20,000 epochs DEFAULT PARAMETERS FOR THRESHOLDS

In [11]:
gen = Generator(z_dim = 107)
load_model(gen, "3 Label 4 Subject GAN Ablation_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=107, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=37, bias=True)
    (4): Tanh()
  )
)

In [54]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 4)

fake_labels = []

for k in range(size):
    if act_vectors[0][k] == 0 and usr_vectors[0][k] == 0:
        fake_labels.append(0)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 0:
        fake_labels.append(1)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 0:
        fake_labels.append(2)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 1:
        fake_labels.append(3)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 1:
        fake_labels.append(4)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 1:
        fake_labels.append(5)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 2:
        fake_labels.append(6)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 2:
        fake_labels.append(7)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 2:
        fake_labels.append(8)
    elif act_vectors[0][k] == 0 and usr_vectors[0][k] == 3:
        fake_labels.append(9)
    elif act_vectors[0][k] == 1 and usr_vectors[0][k] == 3:
        fake_labels.append(10)
    elif act_vectors[0][k] == 2 and usr_vectors[0][k] == 3:
        fake_labels.append(11)
        
fake_labels = np.asarray(fake_labels)
to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()
#fake_features = gen(to_gen).detach().numpy()

In [55]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix((fake_labels), preds.cpu()))
print(metrics.classification_report((fake_labels), preds.cpu(), digits = 5))

[[14  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 15  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  2  0  0  3]
 [ 0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  0  2  0  0 13  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 10]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        14
           1    1.00000   1.00000   1.00000        12
           2    1.00000   1.00000   1.00000        11
           3    1.00000   1.00000   1.00000        15
           4    1.00000   0.90909   0.95238        11
           5    0.00000   0.00000   0.00000         5
           6    1.00000   1.00000   1.00000        12
           7    0.71429   1.00000   0.83333         5
           8    0.00000   0.00000   0.00000 